In [3]:
import os
from langame_client import LangameClient
import random
import openai
from firebase_admin import firestore
#from sentence_transformers import SentenceTransformer
#import numpy as np
#import faiss
import glob

c = LangameClient()

In [4]:
memes = []
for e in c._firestore_client.collection('memes').stream():
    memes.append((e.id, e.to_dict()))

In [5]:
len(memes)

707

In [6]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3
def get_prompt(topics):
    rnd = random.choices(memes, k=500)
    return [(e[1]['content'], e[1]['topics']) for e in rnd if len(intersection(e[1]['topics'], topics)) > 0]

In [60]:
from tqdm import tqdm
import time
from random import randint
import shutil

outputs = []
for i in tqdm(range(10**12)):
  time.sleep(randint(1,3))
  samples = get_prompt(['mathematic', 'physic', 'knowledge', 'artificial intelligence', 'biology'])
  p = list(str(samples[0:60]))
  # replace last char by a ","
  p[-1] = ","
  p = "".join(p)
  try:
    response = openai.Completion.create(
      engine="davinci-codex",
      prompt=p,
      temperature=1,
      max_tokens=100,
      top_p=1,
      frequency_penalty=0.5,
      presence_penalty=0,
      stop=["])"]
    )
  except Exception as e:
    print(e)
    continue
  text = response["choices"][0]["text"] + "]),"
  outputs.append(text)
  # every 10 samples, append the generated samples to file
  if i % 10 == 0:
    with open("generated_samples.txt", "a") as f:
      f.write("\n".join(outputs) + "\n")
    outputs = []
  # icloud = "/Users/louisbeaumont/Library/Mobile Documents/com~apple~CloudDoc/"
  # # every 1000 samples, move file to icloud
  # if i % 10 == 0:
  #   try:
  #     # add ,\n] to the file
  #     with open("generated_samples.txt", "a") as f:
  #       f.write(",\n]")
  #     shutil.move("./generated_samples.txt", icloud + "generated_samples.txt")
  #     os.remove("generated_samples.txt")
  #   except Exception as e:
  #     print(e)

  0%|          | 2666/1000000000000 [2:46:01<1037948634:53:22,  3.74s/it]


KeyboardInterrupt: 

In [8]:
n_m = [
    ("If any kind of AI is supposedly great, where would it have its main limitation? Would that limit depend on government regulation or poor corresponding laws?",['artificial intelligence']),
    ("Quantum Entanglement solves many problems like energy creation without more physical material than can be at most one atom apart...So I read it. Can you give me some proofs?", ['physic']),
    ('Is what comes out of Newton\'s prism true even if this was cast aside in favour of quantum mechanics & quantized reality. Quantum physics under certain circumstances allow relativity theory, which is not classical can it be seen as contradictory or are space-time constantly being disrupted by E = mc^2.',['mathematic']),
    ("I could say that AI will ‘blow us out of the water’ in terms of computational power by 2030.. but have you considered what are its technological requirements ?", ['physic']),
    ('Do we own our identity and thoughts or is there more we don’t know?',['philosophy','mind']),
    ('Do you think the human brain will ever be quantified ?', ['mathematic']),
    ('Does language affect the ways that one thinks and how do you think differently simply because of this. Is there a way to weigh pros and cons without basing onesifications on religious influence i suppose.',['artificial intelligence']),
    ('Will computers inevitably simulate consciousness in some wierd way. That might be induced by something else that you have no idea about…. Are certain words semantics ad information theory which are being processed not neural connections ?\n .. what was “bold” to schopenhauer when introducing the subject of change , he saw it as ridiculous.',['artificial intelligence'] ),
    ('If all things are continuous do not exist what makes them separate ?', ['mathematic','philosophy']),
    ('What would happen given that physcals design our machines to last forever. was it the moral conduct or physical predilection for discovery? ',['physic']),
    ('What happens inside the black hole?',['physic','mathematic']),
    ("stars are dying, does that mean something musn't you think?",['knowledge']),
    ('Is there any indication about how our universe will end ?',['physic','mathematic']),
    ("can machines earn ethics. How should't they be designed to do that first?", ['artificial intelligence', 'knowledge'] ),
    ("If you wanted a useful definition of 'intelligence', precisely how would one quantify all combination abilities? For example, I have the experience with 2 numbers being added but not able to sing along - this kind/experience can it exist or does nature deny us for any complex intuitive processes.",['mind', 'psychology']),
    ("Does society control what we do with our time?", ["mind"]),
    ('What can you really expect from a logical consistent theory?', ['philosophy']),
    ('Can human existence be explained by chance alone, is there any conceivable explanation for nature ?', ["biology"]),
    ('What happens if our environment gets so polluted the only way we can survive is by destroying ourselves?', ["biology"]),
    ('What is the point of mathematics?', ['mathematic', 'knowledge', 'philosophy']),
    ('Do you think we will still use the same mathematics when we merge with machines?', ['mathematic', 'knowledge', 'transhumanism', 'artificial intelligence']),
    ('If you had unlimited knowledge, what would your life look like?', ['knowledge', 'philosophy', 'ice breaker']),
    ('Is there anything that can be mathematically described but not understood with our intelligence?', ['mathematic']),
    ("Which subjects would you prefer to receive a PhD in if there was no limit on your choice of subject?",  ['knowledge', 'philosophy', 'psychology']),
    ("Which fictional character do you most identify with ?", ['ice breaker']),
]

In [9]:
def is_valid():
    invalids = []
    for e in n_m:
        if not isinstance(e[0], str) or not isinstance(e[1], list):
            invalids.append(e)
    return invalids

In [10]:
is_valid()

[]

In [51]:
for e in n_m:
    d = {
        'content': e[0], 
        'topics': e[1],
        'disabled': False,
        'createdAt': firestore.SERVER_TIMESTAMP,
        'tweet': True,
    }
    c._firestore_client.collection('memes').add(d)

In [ ]:
sentences = ["How will humans distinguish themselves from an intelligent machine", "Is artificial intelligence an expression of human nature or does it violate basic principles that the programming was based on ?"]

model = SentenceTransformer('sentence-transformers/paraphrase-xlm-r-multilingual-v1')
embeddings = model.encode(sentences)
print(embeddings)

In [ ]:
model = SentenceTransformer('all-mpnet-base-v2')
embeddings = [model.encode(e[1]['content']) for e in memes]

In [ ]:
np.save("embeddings/part1.npy", embeddings)

In [ ]:
quantizer = Quantizer()
quantizer.quantize(embeddings_path="embeddings", output_path="my_index_folder", max_index_memory_usage="4G", current_memory_available="4G")

In [ ]:


my_index = faiss.read_index(glob.glob("my_index_folder/*.index")[0])

query_vector = np.float32(np.random.rand(1, 100))
k = 5
distances, indices = my_index.search(query_vector, k)

print(list(zip(distances[0], indices[0])))
